# 1.Load data

In [2]:
import keras
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.python.keras.backend import get_session

In [3]:
import pandas as pd
import imblearn
import numpy as np

In [4]:
data = pd.read_csv('./OUTPUT/dataset.csv', encoding= 'unicode_escape')
data

,Sentence,Word,Tag
0,Sentence: 1,RECORD,0
1,Sentence: 2,OC,0
2,NaN,AM,0
3,NaN,gallstone,0
4,NaN,pancreatitis,0
...,...,...,...
949802,NaN,M.D.,0
949803,Sentence: 132094,END,0
949804,NaN,OF,0
949805,NaN,DISCHARGE,0


## Groupby

In [5]:
data_fillna = data.fillna(method='ffill', axis=0)
data_group = data_fillna.groupby(['Sentence'],as_index=False
                                )['Word', 'Tag'].agg(lambda x: list(x))

#data_fillna
data_group

<ipython-input-5-73aff18756c8>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_group = data_fillna.groupby(['Sentence'],as_index=False


,Sentence,Word,Tag
0,Sentence: 1,[RECORD],[0]
1,Sentence: 10,"[WILL, D/C, ORDER, BE, USED, AS, THE, D/C, SUM...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,Sentence: 100,"[prandial, N/V/severe, upper, abdominal, pain....","[0, 1, 0, 1, 1, 0, 0, 0, 0]"
3,Sentence: 1000,"[normal, limits., Cardiac, catheterization, da...","[0, 0, 0, 0, 0, 0, 0, 0]"
4,Sentence: 10000,"[year, old, Black, female, with, significant, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...
132089,Sentence: 99995,"[Height, foot, inch, and, weight, kg., Tempera...","[0, 0, 0, 0, 0, 0, 0]"
132090,Sentence: 99996,"[degrees, heart, rate, and, sinus, blood, pres...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
132091,Sentence: 99997,"[blood, pressure, left, arm, and, oxygen, satu...","[0, 0, 0, 0, 0, 0, 0]"
132092,Sentence: 99998,"[No, carotid, bruits, regular, rate, and, rhyt...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"


## Padding

In [6]:
texts = data_group['Word'].tolist()  
labels = data_group['Tag'].tolist()  

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

pad_tokens = pad_sequences(sequences, maxlen=49, dtype='int32', padding='post', value= 0)
print(pad_tokens)
pad_tags = pad_sequences(labels, maxlen=49, dtype='int32', padding='post', value= 0)
train_tokens, test_tokens, train_tags, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.3, train_size=0.7, random_state=2020)
print(pad_tags)
print('Shape of data tensor:', pad_tokens.shape)
print('Shape of label tensor:', pad_tags.shape)

Found 34275 unique tokens.
[[  115     0     0 ...     0     0     0]
 [   44   145   106 ...     0     0     0]
 [ 6315 15212   259 ...     0     0     0]
 ...
 [   42    70    33 ...     0     0     0]
 [   13   421  1398 ...     0     0     0]
 [  327   561  1373 ...     0     0     0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Shape of data tensor: (132094, 49)
Shape of label tensor: (132094, 49)


## GLOVE Embedding Layer

In [17]:
import os

embeddings_index = {}
f = open(os.path.join(r'./GLOVE', 'glove.6B.300d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [18]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [19]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            300,
                            weights=[embedding_matrix],
                            input_length=49,
                            trainable=True)

## Model

In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
from keras import backend as K
import keras as keras
from keras.datasets import mnist
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from keras import metrics
from sklearn.utils import class_weight

In [1]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



In [22]:
def get_bilstm_lstm_model():
    model = Sequential()

    # Add Embedding layer
    model.add(embedding_layer)
    # Add bidirectional LSTM
    model.add(Bidirectional(LSTM(units=32, return_sequences=True, dropout=0.5, recurrent_dropout=0.5), merge_mode = 'concat'))

    
    # Add LSTM
    model.add(LSTM(units=32, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))
    model.add(LSTM(units=32, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))
    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(1, activation="sigmoid")))
    
    #Optimiser 
    adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
    
    # Compile model
    model.compile(loss='binary_crossentropy', sample_weight_mode="temporal", optimizer='adam', metrics=['acc', precision_m, recall_m, f1_m])
    model.summary()
    
    return model

In [23]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 999)

fold_number = 1
f1_per_fold = []
recall_per_fold = []
precision_per_fold = []
acc_per_fold = []
loss_per_fold = []

for i, (train_index, val_index) in enumerate(skf.split(pad_tokens, np.sum(pad_tags, axis = 1))):
    
    print("Training on fold " + str(i+1) + "/10..........")
    
    #Split training set and validation set
    x_train, x_val = pad_tokens[train_index], pad_tokens[val_index]
    y_train, y_val = pad_tags[train_index], pad_tags[val_index]
    
    #Assigning sample weights in training set
    print(str(fold_number) + ": started assigning sample weights")
    weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(np.ravel(y_train,order='C')),
                                                 np.ravel(y_train,order='C'))
    
    train_tags2 = np.copy(y_train)
    train_tokens2 = np.copy(x_train)
    train_tags2 = train_tags2.astype(float)
    
    indexTotal = 0
    for tags in train_tags2:
        indexTags = 0
        for symptom in tags:
            if symptom == 1:
                train_tags2[indexTotal][indexTags] = float(weights[1]+5.00)
            else:
                train_tags2[indexTotal][indexTags] = float(weights[0])
            indexTags = indexTags+1
        indexTotal = indexTotal + 1
   
    weights = train_tags2.reshape((-1, 49, 1))
    print(str(fold_number) + ": finished assigning sample weights")
    
    #Getting Model Architecture
    model = get_bilstm_lstm_model()
    
    #Running Model
    history = model.fit(x_train, y_train, sample_weight = weights, batch_size=64, verbose=1, epochs=20)
    
    #Evaluate model
    scores = model.evaluate(x_val, y_val, verbose = 0)
    print(f'Score for fold {fold_number}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}; {model.metrics_names[2]} of {scores[2]}; {model.metrics_names[3]} of {scores[3]}; {model.metrics_names[4]} of {scores[4]} %')
    f1_per_fold.append(scores[4])
    recall_per_fold.append(scores[3])
    precision_per_fold.append(scores[2])
    acc_per_fold.append(scores[1])
    loss_per_fold.append(scores[0])
    
    #Increase fold number
    fold_number = fold_number + 1

C:\Users\jonat\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


Training on fold 1/10..........
1: started assigning sample weights


C:\Users\jonat\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 0 0 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


1: finished assigning sample weights
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 49, 300)           10282800  
                                                                 
 bidirectional_3 (Bidirectio  (None, 49, 64)           85248     
 nal)                                                            
                                                                 
 lstm_10 (LSTM)              (None, 49, 32)            12416     
                                                                 
 lstm_11 (LSTM)              (None, 49, 32)            8320      
                                                                 
 time_distributed_3 (TimeDis  (None, 49, 1)            33        
 tributed)                                                       
                                                                 
Total params: 10,

1858/1858 [==============================] - 620s 334ms/step - loss: 0.0017 - acc: 0.9995 - precision_m: 0.9278 - recall_m: 0.9154 - f1_m: 0.9164
Epoch 9/20
1858/1858 [==============================] - 623s 335ms/step - loss: 0.0016 - acc: 0.9995 - precision_m: 0.9278 - recall_m: 0.9178 - f1_m: 0.9173
Epoch 10/20
1858/1858 [==============================] - 622s 335ms/step - loss: 0.0016 - acc: 0.9995 - precision_m: 0.9270 - recall_m: 0.9173 - f1_m: 0.9168
Epoch 11/20
1858/1858 [==============================] - 620s 334ms/step - loss: 0.0015 - acc: 0.9995 - precision_m: 0.9302 - recall_m: 0.9221 - f1_m: 0.9208
Epoch 12/20
1858/1858 [==============================] - 623s 335ms/step - loss: 0.0015 - acc: 0.9995 - precision_m: 0.9336 - recall_m: 0.9261 - f1_m: 0.9251
Epoch 13/20
1858/1858 [==============================] - 623s 335ms/step - loss: 0.0014 - acc: 0.9995 - precision_m: 0.9335 - recall_m: 0.9270 - f1_m: 0.9249
Epoch 14/20
1858/1858 [==============================] - 624s 336

1858/1858 [==============================] - 747s 392ms/step - loss: 0.0114 - acc: 0.9977 - precision_m: 0.6805 - recall_m: 0.5613 - f1_m: 0.5890
Epoch 2/20
1858/1858 [==============================] - 742s 399ms/step - loss: 0.0034 - acc: 0.9990 - precision_m: 0.8573 - recall_m: 0.8531 - f1_m: 0.8441
Epoch 3/20
1858/1858 [==============================] - 745s 401ms/step - loss: 0.0025 - acc: 0.9992 - precision_m: 0.8958 - recall_m: 0.8897 - f1_m: 0.8851
Epoch 4/20
1858/1858 [==============================] - 748s 403ms/step - loss: 0.0021 - acc: 0.9994 - precision_m: 0.9090 - recall_m: 0.9000 - f1_m: 0.8980
Epoch 5/20
1858/1858 [==============================] - 739s 398ms/step - loss: 0.0018 - acc: 0.9994 - precision_m: 0.9182 - recall_m: 0.9087 - f1_m: 0.9073
Epoch 6/20
1858/1858 [==============================] - 740s 398ms/step - loss: 0.0017 - acc: 0.9995 - precision_m: 0.9281 - recall_m: 0.9167 - f1_m: 0.9169
Epoch 7/20
1858/1858 [==============================] - 738s 397ms/st

C:\Users\jonat\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 1 0 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


8: finished assigning sample weights
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 49, 300)           10282800  
                                                                 
 bidirectional_10 (Bidirecti  (None, 49, 64)           85248     
 onal)                                                           
                                                                 
 lstm_31 (LSTM)              (None, 49, 32)            12416     
                                                                 
 lstm_32 (LSTM)              (None, 49, 32)            8320      
                                                                 
 time_distributed_10 (TimeDi  (None, 49, 1)            33        
 stributed)                                                      
                                                                 
Total params: 10

In [24]:
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print("-----------")
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]} - Precision: {precision_per_fold[i]} - Recall: {recall_per_fold[i]} - F1: {f1_per_fold[i]}%')
print('------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Precision: {np.mean(precision_per_fold)} (+- {np.std(precision_per_fold)})')
print(f'> Recall: {np.mean(recall_per_fold)} (+- {np.std(recall_per_fold)})')
print(f'> F1: {np.mean(f1_per_fold)} (+- {np.std(f1_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------')

Score per fold
-----------
> Fold 1 - Loss: 0.008424668572843075 - Accuracy: 0.9981651306152344 - Precision: 0.7087938189506531 - Recall: 0.669576108455658 - F1: 0.6605508327484131%
-----------
> Fold 2 - Loss: 0.004767275415360928 - Accuracy: 0.9988278746604919 - Precision: 0.7777432799339294 - Recall: 0.7450389266014099 - F1: 0.744812548160553%
-----------
> Fold 3 - Loss: 0.0038301546592265368 - Accuracy: 0.9990360140800476 - Precision: 0.7775216102600098 - Recall: 0.7582077980041504 - F1: 0.7497233748435974%
-----------
> Fold 4 - Loss: 0.003117585787549615 - Accuracy: 0.9991922974586487 - Precision: 0.8177358508110046 - Recall: 0.791504442691803 - F1: 0.7899191379547119%
-----------
> Fold 5 - Loss: 0.002637598430737853 - Accuracy: 0.9993019104003906 - Precision: 0.8417547345161438 - Recall: 0.8206593990325928 - F1: 0.8203046917915344%
-----------
> Fold 6 - Loss: 0.0025533344596624374 - Accuracy: 0.9993559718132019 - Precision: 0.8410406708717346 - Recall: 0.8165351748466492 - F1

## Self Check

In [74]:
predict_tags = model.predict(test_tokens)

In [75]:
truePositive = 0 
falsePositive = 0
trueNegative = 0 
falseNegative = 0
index = 0
for tag, predTag in zip(test_tags, predict_tags):
    for symptomTag, symptomPred in zip (tag, predTag):
        if symptomPred >= 0.50 and symptomTag == 1:
            truePositive = truePositive + 1
        elif symptomPred >= 0.50 and symptomTag == 0:
            falsePositive = falsePositive + 1
        elif symptomPred < 0.50 and symptomTag == 0:
            trueNegative = trueNegative + 1
        elif symptomPred < 0.50 and symptomTag == 1:
            falseNegative = falseNegative + 1

In [76]:
print('True postitive: ' + str(truePositive))
print('False postitive: ' + str(falsePositive))
print('True negative: ' + str(trueNegative))
print('False negative: ' + str(falseNegative))

True postitive: 6887
False postitive: 359
True negative: 1934322
False negative: 253


In [77]:
precision = (truePositive)/(truePositive+falsePositive)
recall = (truePositive)/(truePositive+falseNegative)

In [78]:
print('Accuracy: ' + str((truePositive+trueNegative)/(truePositive+trueNegative+falsePositive+falseNegative)))
print('Precision: ' + str((truePositive)/(truePositive+falsePositive)))
print('Recall: ' + str((truePositive)/(truePositive+falseNegative)))
print('F1: ' + str((2*precision*recall)/(precision+recall)))

Accuracy: 0.999684831918081
Precision: 0.9504554236820315
Recall: 0.9645658263305322
F1: 0.9574586403447796


In [79]:
model.evaluate(pad_tokens, pad_tags)

4128/4128 [==============================] - 215s 52ms/step - loss: 0.0010 - acc: 0.9997 - precision_m: 0.6882 - recall_m: 0.6997 - f1_m: 0.6890


[0.0010336712002754211,
 0.9996744990348816,
 0.6881966590881348,
 0.6996971368789673,
 0.6890236735343933]